# Search Tester Analysis
*Follow these steps to compare legacy results to the new api results.*

### Setup the environment

In [1]:
%run ./local/setup.ipynb

Unable to configure logging, attempted conf:/home/notebook/.local/lib/python3.10/site-packages/ppr_api/logging.conf


### Find search batches (choose the data set)
*find the ones you want to analyze*

In [3]:
search_type = None
after_date = None
before_date = None
### OPTIONAL: refine your batch search with any of the following
# search_type = SearchRequest.SearchTypes.XXXX
# before_date = _datetime_in_utc_
after_date = "2022-01-22 17:48:30.936128"
batches = TestSearchBatch.find_search_batches(search_type, after_date, before_date)
for batch in batches:
    print(batch.id, batch.test_date, batch.search_type, batch.sim_val_business, batch.sim_val_first_name, batch.sim_val_last_name)


831 2022-01-22 18:25:01.968199 IS 0.6 0.4 0.29


In [4]:
# add the ids of the batches you want to analyze
batch_ids = []
for batch in batches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. batch.id > _ , batch.sim_val_business > _ ...
        batch_ids.append(batch.id)

### Get the batch json

In [5]:
batches_to_analyze = []
for batch_id in batch_ids:
    batches_to_analyze.append(TestSearchBatch.find_by_id(batch_id))

# Analysis

### Exact Matches

#### Pass rate

In [6]:
searches_total = 0
searches_passed = 0
failed_searches = []
all_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        all_searches.append(search)
        if len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.9598765432098766
number of failed searches:  13


#### Failed Searches

*select failed searches to analyze*

In [7]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesExact']['resultsLegacy']), 'missed: ', len(json['matchesExact']['missedMatches']))

###########################################################
criteria:  YEE                      WESLEY         T                             
total expected:  1 missed:  1
###########################################################
criteria:  CENTER                   C              K                             
total expected:  2 missed:  2
###########################################################
criteria:  SMITH                    BENJAMIN       JAMES                         
total expected:  10 missed:  1
###########################################################
criteria:  BROWN                    ROSEMARY                                     
total expected:  1 missed:  1
###########################################################
criteria:  CHEN                     BENSON                                       
total expected:  6 missed:  5
###########################################################
criteria:  LAM                      ANNETTE                                    

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [8]:
exact_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > _
        exact_searches_analysis.append(search)

*missed matches*

In [9]:
import re
### manually iterate through exact_searches_analysis one by one to see the missed matches
# search = exact_searches_analysis[1]
# for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])
### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  YEE                      WESLEY         T                             
Missed Matches
-------------------------------------------------------------------------------
result: YEE                      W                              72AUG24
reg num: 309732K
index:  0
##################################################################################
criteria:  CENTER                   C              K                             
Missed Matches
-------------------------------------------------------------------------------
result: CENTER                   CHRISTOPHER    KENT            75DEC06
reg num: 769800I
index:  0
-------------------------------------------------------------------------------
result: CENTER                   CHRISTOPHER    KENT            75DEC06
reg num: 553828M
index:  1
##################################################################################
criteria:  SMITH    

*results diff*

In [9]:
import re
### manually iterate through exact_searches_analysis one by one
# search = exact_searches_analysis[1]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('-------------------------------------------------------------')
    print('  legacy                           api')
    print('-------------------------------------------------------------')
    legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
    api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
    length = max(len(legacy_results), len(api_results))
    for i in range(length):
        if i < len(legacy_results) and i < len(api_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
        elif i < len(legacy_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
        elif i < len(api_results):
            print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')


### Similar Matches

#### Pass rate

In [10]:
searches_total = 0
searches_passed = 0
failed_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        # fails if missed matches
        if len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > 0:
            failed_searches.append(search)
        # fails if order is off
        elif search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value) != 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.33641975308641975
number of failed searches:  215


#### Failed searches

*select failed searches to analyze*

In [11]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesSimilar']['resultsLegacy']), ' missed: ', len(json['matchesSimilar']['missedMatches']))
    print('first fail index: ', json['matchesSimilar']['firstFailIndex'])
    print('avg index diff: ', json['matchesSimilar']['avgIndexDiff'])

###########################################################
criteria:  DR. CHRISTOPHER RICKARDS INC                                          
total expected:  6  missed:  6
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  GREENER SPACES                                                        
total expected:  20  missed:  0
first fail index:  0
avg index diff:  10.6
###########################################################
criteria:  DIG IT CONTRACTING                                                    
total expected:  22  missed:  22
first fail index:  0
avg index diff:  0
###########################################################
criteria:  C & D CAPITAL                                                         
total expected:  3  missed:  3
first fail index:  0
avg index diff:  0
###########################################################
criteria:  JB OFFICE SYSTEMS                                                     

###########################################################
criteria:  J.ROSE CORPORATION                                                    
total expected:  2  missed:  2
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  THE OWNERS STRATA PLAN EPS 6752                                       
total expected:  10  missed:  1
first fail index:  0
avg index diff:  31.5
###########################################################
criteria:  DREAM VILLA                                                           
total expected:  1  missed:  0
first fail index:  0
avg index diff:  1.0
###########################################################
criteria:  KALIA HOLDINGS LTD                                                    
total expected:  13  missed:  13
first fail index:  0
avg index diff:  0
###########################################################
criteria:  DR. SUMEET KALIA INC.                                               

###########################################################
criteria:  STRAIT PIPELINE & INTEGRITY LTD.                                      
total expected:  1  missed:  0
first fail index:  0
avg index diff:  72.0
###########################################################
criteria:  PIZZA PIZZA LIMITED                                                   
total expected:  2  missed:  2
first fail index:  0
avg index diff:  0
###########################################################
criteria:  CARLETON GROUP INC.                                                   
total expected:  1  missed:  1
first fail index:  0
avg index diff:  0
###########################################################
criteria:  NEXT LEVEL CHILDCARE VALLEY FAIR INC.                                 
total expected:  6  missed:  6
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  MISSION GROUP INC.                                                    
to

###########################################################
criteria:  WHITE DRYWALL & DEVELOPMENT LTD                                       
total expected:  4  missed:  3
first fail index:  0
avg index diff:  2.0
###########################################################
criteria:  APNA DRYWALL & CONSTRUCTION LTD                                       
total expected:  1  missed:  1
first fail index:  0
avg index diff:  0
###########################################################
criteria:  RYAN COMPANY LIMITED                                                  
total expected:  3  missed:  3
first fail index:  0
avg index diff:  0
###########################################################
criteria:  NORTH SHORE BETTER                                                    
total expected:  1  missed:  1
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  LOON PROPERTIES INC.                                                  
tot

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [11]:
similar_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > _
        similar_searches_analysis.append(search)

*first failed indexes*

In [11]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

0


In [7]:
### avgs
total_no_fails = 0
total_fail_index = 0
for search in similar_searches_analysis:
    fail_index = search.fail_index(TestSearchResult.MatchType.SIMILAR.value)
    if fail_index == -1:
        total_no_fails += 1
    else:
        total_fail_index += fail_index

num_searches_failed = len(similar_searches_analysis) - total_no_fails
print('Number of searches with fail indexes: ', num_searches_failed)
print('avg fail index: ', total_fail_index/num_searches_failed)

Number of searches with fail indexes:  608
avg fail index:  0.32894736842105265


*avg order difference between legacy and api results (does NOT include missed matches)*

In [8]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

0


*missed matches*

In [12]:
import re
### manually iterate through
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('Missed Matches')
# for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])

### print out all of them
for search in similar_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  FERGUSON                 GREGARY                                      
Missed Matches
##################################################################################
criteria:  TIDD                     ROBERT                                       
Missed Matches
-------------------------------------------------------------------------------
result: TADA                     R                              71OCT29
reg num: 521940M
index:  0
-------------------------------------------------------------------------------
result: TADDEI                   ROBERT         J               84JAN13
reg num: 842888L
index:  1
-------------------------------------------------------------------------------
result: TADDEI                   ROBERT-JO      DOMENIC         84JAN13
reg num: 842089J
index:  2
-------------------------------------------------------------------------------
result: THIRD          

-------------------------------------------------------------------------------
result: INGERSOLL                KELLY                          81OCT04
reg num: 522018J
index:  0
-------------------------------------------------------------------------------
result: INGERSOLL                KELLY          PAIGE           81OCT04
reg num: 522018J
index:  1
-------------------------------------------------------------------------------
result: INGRAM                   KAYLIE         WOOD            74OCT20
reg num: 177533J
index:  3
##################################################################################
criteria:  MACKENZIE                TAYLOR                                       
Missed Matches
##################################################################################
criteria:  BROWN                    RYAN                                         
Missed Matches
-------------------------------------------------------------------------------
result: BRAUN          

-------------------------------------------------------------------------------
result: LAI                      LIN                            54MAR14
reg num: 331423L
index:  0
-------------------------------------------------------------------------------
result: LAI                      LINDA                          76JUN22
reg num: 436716M
index:  1
-------------------------------------------------------------------------------
result: LAU                      LINDA                          81SEP16
reg num: 976146L
index:  2
-------------------------------------------------------------------------------
result: LO                       LIN                            79NOV26
reg num: 655370L
index:  11
-------------------------------------------------------------------------------
result: LUU                      LINDA                          82APR30
reg num: 525946K
index:  12
##################################################################################
criteria:  ZHONG    

-------------------------------------------------------------------------------
result: LEE                      MIN                            82FEB26
reg num: 979514K
index:  1
##################################################################################
criteria:  CHAN                     WILLIAM                                      
Missed Matches
-------------------------------------------------------------------------------
result: CHAN                     BILLY                          85JAN10
reg num: 982270L
index:  2
-------------------------------------------------------------------------------
result: CHAN                     BILLY                          85JAN10
reg num: 327591M
index:  3
-------------------------------------------------------------------------------
result: CHAN                     BILLY                          86AUG14
reg num: 469268L
index:  4
-------------------------------------------------------------------------------
result: CHAN            

-------------------------------------------------------------------------------
result: LEE                      LAIKUEN                        67MAR02
reg num: 367308L
index:  0
##################################################################################
criteria:  LEE                      CINDY          L                             
Missed Matches
-------------------------------------------------------------------------------
result: LEE                      LAIKUEN                        67MAR02
reg num: 367308L
index:  0
##################################################################################
criteria:  VINCE                    STEVEN         E                             
Missed Matches
-------------------------------------------------------------------------------
result: VANECK                   STEVEN                         75NOV08
reg num: 672478L
index:  0
##################################################################################
criteria:  LU       

...

-------------------------------------------------------------------------------
result: CHAN                     CHAI                           59JAN06
reg num: 840633L
index:  0
-------------------------------------------------------------------------------
result: CHAN                     CHE                            56APR07
reg num: 402645L
index:  1
-------------------------------------------------------------------------------
result: CHAN                     CHI                            51NOV27
reg num: 874103L
index:  2
-------------------------------------------------------------------------------
result: CHAN                     CHI                            59FEB16
reg num: 661976L
index:  3
-------------------------------------------------------------------------------
result: CHAN                     CHI                            61SEP21
reg num: 574851L
index:  4
-------------------------------------------------------------------------------
result: CHAN             

-------------------------------------------------------------------------------
result: SMITH                    DARELL         DANIEL          64SEP05
reg num: 874601L
index:  0
##################################################################################
criteria:  MCLEOD                   DWAYNE                                       
Missed Matches
-------------------------------------------------------------------------------
result: MCLEOD                   DEAN           KELLY           67MAY31
reg num: 656960C
index:  2
-------------------------------------------------------------------------------
result: MCLEOD                   DEAN           STEWART         66FEB28
reg num: 477886M
index:  3
-------------------------------------------------------------------------------
result: MCLEOD                   DIANA          MARIA           74AUG06
reg num: 293477M
index:  4
-------------------------------------------------------------------------------
result: MCLOUDE         

##################################################################################
criteria:  RIGGS                    KRISTA                                       
Missed Matches
-------------------------------------------------------------------------------
result: REAGH                    CHRISTINE      EVA             58DEC21
reg num: 8680669
index:  0
-------------------------------------------------------------------------------
result: REGO                     CHRISTINE      MARGARET        75FEB16
reg num: 264924M
index:  1
##################################################################################
criteria:  SANDHU                   BALJIT                                       
Missed Matches
-------------------------------------------------------------------------------
result: SANDHO                   BALJEET        K               71SEP09
reg num: 628994K
index:  0
-------------------------------------------------------------------------------
result: SANDHO         

-------------------------------------------------------------------------------
result: BRAUN                    STEPHEN                        62FEB07
reg num: 024494J
index:  0
-------------------------------------------------------------------------------
result: BRAWN                    STEPHANIE                      90MAY29
reg num: 927912K
index:  2
-------------------------------------------------------------------------------
result: BREEN                    STEPHEN                        80OCT25
reg num: 374196M
index:  4
-------------------------------------------------------------------------------
result: BREM                     STEFANIE                       85JAN13
reg num: 559430M
index:  5
-------------------------------------------------------------------------------
result: BRIND-SHERIDAN           STEPHANIE                      38APR20
reg num: 670398C
index:  6
-------------------------------------------------------------------------------
result: BROWN            

##################################################################################
criteria:  BIFFERT                  STEVEN                                       
Missed Matches
-------------------------------------------------------------------------------
result: BOUFFARD                 STEFANIE                       85MAR10
reg num: 133762K
index:  0
-------------------------------------------------------------------------------
result: BYFORD                   STEVE          J               76FEB19
reg num: 497514M
index:  1
-------------------------------------------------------------------------------
result: BYFORD                   STEVEN                         65SEP19
reg num: 054874M
index:  2
-------------------------------------------------------------------------------
result: BYFORD                   STEVEN         J               76FEB19
reg num: 497514M
index:  3
-------------------------------------------------------------------------------
result: BYFORD          

*results diff*

In [13]:
missed = 0
for search in similar_searches_analysis:
    if search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        missed += 1

print('searches with missed matches: ', missed)
print('percent of total that do not have a missed similar: ', (searches_total-missed)/searches_total)

print('total searches: ', searches_total)

# import re
# ### manually iterate through similar_searches_analysis one by one
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
# for search in similar_searches_analysis:
#     print('criteria: ', search.search_criteria)
#     print('-------------------------------------------------------------')
#     print('  legacy                           api')
#     print('-------------------------------------------------------------')
#     legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
#     api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
#     length = max(len(legacy_results), len(api_results))
#     for i in range(length):
#         if i < len(legacy_results) and i < len(api_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#         elif i < len(legacy_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#         elif i < len(api_results):
#             print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

searches with missed matches:  167
percent of total that do not have a missed similar:  0.4845679012345679
total searches:  324


### Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [18]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2